### Preprocessing

In [1]:
# import packages
import numpy as np
import pandas as pd
import statsmodels.api as sm
from statsmodels.formula.api import ols, Logit
from scipy import stats
import matplotlib.pyplot as plt
pd.options.display.float_format = '{:,}'.format
import seaborn as sns
%matplotlib inline

In [21]:
# import data
url = "/Users/arpanganguli/Documents/Professional/Finance/ISLR/Wage.csv"
Wage = pd.read_csv(url, index_col='SlNo')

In [22]:
Wage.head()

,year,age,maritl,race,education,region,jobclass,health,health_ins,logwage,wage
SlNo,,,,,,,,,,,
231655,2006,18,1. Never Married,1. White,1. < HS Grad,2. Middle Atlantic,1. Industrial,1. <=Good,2. No,4.318063334962759,75.0431540173515
86582,2004,24,1. Never Married,1. White,4. College Grad,2. Middle Atlantic,2. Information,2. >=Very Good,2. No,4.25527250510331,70.47601964694451
161300,2003,45,2. Married,1. White,3. Some College,2. Middle Atlantic,1. Industrial,1. <=Good,1. Yes,4.8750612633917,130.982177377461
155159,2003,43,2. Married,3. Asian,4. College Grad,2. Middle Atlantic,2. Information,2. >=Very Good,1. Yes,5.041392685158231,154.68529299563
11443,2005,50,4. Divorced,1. White,2. HS Grad,2. Middle Atlantic,2. Information,1. <=Good,1. Yes,4.318063334962759,75.0431540173515


In [23]:
Wage.describe().round(2)

,year,age,logwage,wage
count,"3,000.0","3,000.0","3,000.0","3,000.0"
mean,"2,005.79",42.41,4.65,111.7
std,2.03,11.54,0.35,41.73
min,"2,003.0",18.0,3.0,20.09
25%,"2,004.0",33.75,4.45,85.38
50%,"2,006.0",42.0,4.65,104.92
75%,"2,008.0",51.0,4.86,128.68
max,"2,009.0",80.0,5.76,318.34


In [24]:
Wage.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3000 entries, 231655 to 453557
Data columns (total 11 columns):
year          3000 non-null int64
age           3000 non-null int64
maritl        3000 non-null object
race          3000 non-null object
education     3000 non-null object
region        3000 non-null object
jobclass      3000 non-null object
health        3000 non-null object
health_ins    3000 non-null object
logwage       3000 non-null float64
wage          3000 non-null float64
dtypes: float64(2), int64(2), object(7)
memory usage: 281.2+ KB


### GAMs

In [220]:
from pygam import LinearGAM, LogisticGAM, s, f
from patsy import dmatrix

In [271]:
X = pd.concat([Wage['year'], Wage['age'], Wage['education'].astype('category').cat.codes], axis=1)
X.rename(columns={0: 'education'}, inplace=True)
y = Wage['wage']

In [273]:
X.head()

,year,age,education
SlNo,,,
231655,2006,18,0
86582,2004,24,3
161300,2003,45,2
155159,2003,43,3
11443,2005,50,1


In [274]:
X_age = dmatrix("cr(AGE, df=5)", {"AGE": Wage['age']}, return_type='dataframe')
X_year = dmatrix("cr(YEAR, df=4)", {"YEAR": Wage['year']}, return_type='dataframe')
X_education = Wage['education']
y = Wage.wage
df = pd.concat([y, X_year, X_age, X_education], axis=1)
df.head()

,wage,Intercept,"cr(YEAR, df=4)[0]","cr(YEAR, df=4)[1]","cr(YEAR, df=4)[2]","cr(YEAR, df=4)[3]",Intercept,"cr(AGE, df=5)[0]","cr(AGE, df=5)[1]","cr(AGE, df=5)[2]","cr(AGE, df=5)[3]","cr(AGE, df=5)[4]",education
SlNo,,,,,,,,,,,,,
231655,75.0431540173515,1.0,-0.07500000000000001,0.575,0.575,-0.07500000000000001,1.0,1.0,0.0,0.0,0.0,0.0,1. < HS Grad
86582,70.47601964694451,1.0,0.4,0.725,-0.15000000000000002,0.025,1.0,0.5100266666666666,0.60384,-0.14336000000000002,0.03443450980392157,-0.004941176470588235,4. College Grad
161300,130.982177377461,1.0,1.0,0.0,0.0,0.0,1.0,-0.030933333333333337,0.19359999999999997,0.9216000000000001,-0.0983843137254902,0.014117647058823532,3. Some College
155159,154.68529299563,1.0,1.0,0.0,0.0,0.0,1.0,-0.05300411522633745,0.3550617283950617,0.8019753086419753,-0.12146211571048174,0.017429193899782137,4. College Grad
11443,75.0431540173515,1.0,0.0,1.0,0.0,0.0,1.0,0.013156213991769547,-0.07893728395061728,0.9667120987654322,0.11526940692326314,-0.016200435729847498,2. HS Grad


In [275]:
gam1 = ols('y~X_year+X_age+X_education', data=df).fit()
gam1.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.292
Model:                            OLS   Adj. R-squared:                  0.289
Method:                 Least Squares   F-statistic:                     111.8
Date:                Wed, 23 Jan 2019   Prob (F-statistic):          2.63e-214
Time:                        23:11:11   Log-Likelihood:                -14933.
No. Observations:                3000   AIC:                         2.989e+04
Df Residuals:                    2988   BIC:                         2.996e+04
Df Model:                          11                                         
Covariance Type:            nonrobust                                         
=====================================================================================================
                                        coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------
Intercept                            21.8576      0.844     25.892      0.000      20.202      23.513
X_education[T.2. HS Grad]            10.7858      2.431      4.438      0.000       6.020      15.551
X_education[T.3. Some College]       23.1820      2.560      9.055      0.000      18.162      28.202
X_education[T.4. College Grad]       37.8598      2.543     14.889      0.000      32.874      42.846
X_education[T.5. Advanced Degree]    62.3017      2.762     22.556      0.000      56.886      67.717
X_year[0]                            21.8576      0.844     25.892      0.000      20.202      23.513
X_year[1]                             1.0304      1.303      0.791      0.429      -1.525       3.586
X_year[2]                             5.4341      1.184      4.589      0.000       3.112       7.756
X_year[3]                             7.0449      1.233      5.712      0.000       4.627       9.463
X_year[4]                             8.3482      1.447      5.767      0.000       5.510      11.186
X_age[0]                             21.8576      0.844     25.892      0.000      20.202      23.513
X_age[1]                            -25.1395      3.324     -7.563      0.000     -31.657     -18.622
X_age[2]                             12.5790      2.463      5.108      0.000       7.750      17.408
X_age[3]                             21.3751      2.105     10.153      0.000      17.247      25.503
X_age[4]                             16.3377      2.626      6.223      0.000      11.190      21.486
X_age[5]                             -3.2948      7.774     -0.424      0.672     -18.537      11.947
==============================================================================
Omnibus:                     1040.588   Durbin-Watson:                   1.977
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             5586.798
Skew:                           1.557   Prob(JB):                         0.00
Kurtosis:                       8.916   Cond. No.                     2.88e+17
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The smallest eigenvalue is 1.39e-31. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""

In [276]:
X_age_1 = dmatrix("cr(AGE, df=5)", {"AGE": Wage['age']}, return_type='dataframe')
X_year_1 = dmatrix("cr(YEAR, df=4)", {"YEAR": Wage['year']}, return_type='dataframe')
X_education_1 = Wage['education']
y_1 = Wage.wage
df_1 = pd.concat([y_1, X_year_1, X_age_1, X_education_1], axis=1)
df_1.head()

,wage,Intercept,"cr(YEAR, df=4)[0]","cr(YEAR, df=4)[1]","cr(YEAR, df=4)[2]","cr(YEAR, df=4)[3]",Intercept,"cr(AGE, df=5)[0]","cr(AGE, df=5)[1]","cr(AGE, df=5)[2]","cr(AGE, df=5)[3]","cr(AGE, df=5)[4]",education
SlNo,,,,,,,,,,,,,
231655,75.0431540173515,1.0,-0.07500000000000001,0.575,0.575,-0.07500000000000001,1.0,1.0,0.0,0.0,0.0,0.0,1. < HS Grad
86582,70.47601964694451,1.0,0.4,0.725,-0.15000000000000002,0.025,1.0,0.5100266666666666,0.60384,-0.14336000000000002,0.03443450980392157,-0.004941176470588235,4. College Grad
161300,130.982177377461,1.0,1.0,0.0,0.0,0.0,1.0,-0.030933333333333337,0.19359999999999997,0.9216000000000001,-0.0983843137254902,0.014117647058823532,3. Some College
155159,154.68529299563,1.0,1.0,0.0,0.0,0.0,1.0,-0.05300411522633745,0.3550617283950617,0.8019753086419753,-0.12146211571048174,0.017429193899782137,4. College Grad
11443,75.0431540173515,1.0,0.0,1.0,0.0,0.0,1.0,0.013156213991769547,-0.07893728395061728,0.9667120987654322,0.11526940692326314,-0.016200435729847498,2. HS Grad


In [281]:
gam_1 = ols('y~X_year', data=df).fit()
gam_2 = ols('y~X_year+X_age', data=df).fit()
gam_3 = ols('y~X_year+X_age+X_education', data=df).fit() 

In [282]:
anova_table = sm.stats.anova_lm(gam_1, gam_2, gam_3, typ=1)
anova_table.index = anova_table.index+1
anova_table.round(6)

,df_resid,ssr,df_diff,ss_diff,F,Pr(>F)
1,"2,996.0","5,197,451.916955",0.0,nan,nan,nan
2,"2,992.0","4,743,398.97611",4.0,"454,052.940845",91.694864,0.0
3,"2,988.0","3,698,980.875567",4.0,"1,044,418.100543",210.917641,0.0


In [283]:
gam_3.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.292
Model:                            OLS   Adj. R-squared:                  0.289
Method:                 Least Squares   F-statistic:                     111.8
Date:                Wed, 23 Jan 2019   Prob (F-statistic):          2.63e-214
Time:                        23:16:44   Log-Likelihood:                -14933.
No. Observations:                3000   AIC:                         2.989e+04
Df Residuals:                    2988   BIC:                         2.996e+04
Df Model:                          11                                         
Covariance Type:            nonrobust                                         
=====================================================================================================
                                        coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------
Intercept                            21.8576      0.844     25.892      0.000      20.202      23.513
X_education[T.2. HS Grad]            10.7858      2.431      4.438      0.000       6.020      15.551
X_education[T.3. Some College]       23.1820      2.560      9.055      0.000      18.162      28.202
X_education[T.4. College Grad]       37.8598      2.543     14.889      0.000      32.874      42.846
X_education[T.5. Advanced Degree]    62.3017      2.762     22.556      0.000      56.886      67.717
X_year[0]                            21.8576      0.844     25.892      0.000      20.202      23.513
X_year[1]                             1.0304      1.303      0.791      0.429      -1.525       3.586
X_year[2]                             5.4341      1.184      4.589      0.000       3.112       7.756
X_year[3]                             7.0449      1.233      5.712      0.000       4.627       9.463
X_year[4]                             8.3482      1.447      5.767      0.000       5.510      11.186
X_age[0]                             21.8576      0.844     25.892      0.000      20.202      23.513
X_age[1]                            -25.1395      3.324     -7.563      0.000     -31.657     -18.622
X_age[2]                             12.5790      2.463      5.108      0.000       7.750      17.408
X_age[3]                             21.3751      2.105     10.153      0.000      17.247      25.503
X_age[4]                             16.3377      2.626      6.223      0.000      11.190      21.486
X_age[5]                             -3.2948      7.774     -0.424      0.672     -18.537      11.947
==============================================================================
Omnibus:                     1040.588   Durbin-Watson:                   1.977
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             5586.798
Skew:                           1.557   Prob(JB):                         0.00
Kurtosis:                       8.916   Cond. No.                     2.88e+17
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The smallest eigenvalue is 1.39e-31. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""

**Logistic regression GAM**

In [313]:
X_age = dmatrix("cr(AGE, df=5)", {"AGE": Wage['age']}, return_type='dataframe')
X_year = Wage['year']
X_education = Wage['education'].astype('category').cat.codes
y_wage = np.where(Wage['wage']>=250, 1, 0)
dflog = np.array(pd.concat([X_age, X_year, X_education], axis=1))

In [316]:
glmlog = sm.GLM(y_wage, dflog, family=sm.families.Binomial()).fit()
glmlog.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:                      y   No. Observations:                 3000
Model:                            GLM   Df Residuals:                     2993
Model Family:                Binomial   Df Model:                            6
Link Function:                  logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -302.07
Date:                Wed, 23 Jan 2019   Deviance:                       604.15
Time:                        23:58:48   Pearson chi2:                 2.95e+03
No. Iterations:                     9   Covariance Type:             nonrobust
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const        -45.7258     96.565     -0.474      0.636    -234.991     143.539
x1           -14.6003     19.508     -0.748      0.454     -52.835      23.634
x2            -7.1997     19.283     -0.373      0.709     -44.994      30.595
x3            -6.7774     19.277     -0.352      0.725     -44.560      31.005
x4            -6.4585     19.301     -0.335      0.738     -44.288      31.371
x5           -10.6897     19.611     -0.545      0.586     -49.127      27.748
x6             0.0229      0.058      0.397      0.691      -0.090       0.136
x7             1.0727      0.132      8.120      0.000       0.814       1.332
==============================================================================
"""